In [1]:
import numpy as np
import opensim as osim
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import os

In [2]:
# Keeping pandas from truncating long strings
pd.set_option('display.max_colwidth', 10000000)

In [3]:
# ensure correct individual name
cwd = os.getcwd()
root = os.path.dirname(os.path.dirname(os.path.dirname(cwd)))
print(f"Root directory: {root}")
tmp = cwd.split('\\')[len(root.split('\\')):]
ind = 'P' + tmp[0][1:]
ind

Root directory: e:\DataFolder\AlexP\models\athlete_03


'P01bis'

## Update Scaled Model with TPS data

In [4]:
scaled_model = f"3a_osim_markers/scaled_model_{ind}_joints.osim"
new_model_name = os.path.join(cwd, f'4_tps-bones-muscles-updated/{ind}_tps_transformed.osim')

experimental_markers = r"E:\DataFolder\AlexP\simulations\22_07_06\static_01\marker_experimental.trc"

In [5]:
# IMPORTANT : THESE SHOULD BE PERSON-SPECIFIC
mass_text = '100'

height_text = '1.83'

age_text = '30'

# path to experimental .trc file : <marker_file>
static_df = pd.read_csv(experimental_markers, delimiter='\t', skiprows=3, header=[0,1], index_col=0)

### Import Transformed Markers

In [6]:
markers_df = pd.concat([pd.read_csv('tps_warping_results/after_MRI/pelvis_bone_markers.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/femur_l_bone_markers.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/femur_r_bone_markers.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/patella_l_bone_markers.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/patella_r_bone_markers.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/tibia_l_bone_markers.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/tibia_r_bone_markers.csv', index_col=0)])


In [7]:
markers_df = markers_df.set_index('name')
markers_df.to_csv('tps_warping_results/after_MRI/markers_transformed.csv')
print(f'data save to {os.path.join(cwd, "tps_warping_results/after_MRI/markers_transformed.csv")}')

data save to e:\DataFolder\AlexP\models\athlete_03\001bis\P01\model_update\tps_warping_results/after_MRI/markers_transformed.csv


In [8]:
patella_l_in_femur = markers_df.loc['patella_l_in_femur_l']

In [9]:
patella_r_in_femur = markers_df.loc['patella_r_in_femur_r']

### Import Transformed Muscle Paths

In [10]:
mscles_df = pd.concat([pd.read_csv('tps_warping_results/after_MRI/pelvis_muscle_paths.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/femur_l_muscle_paths.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/femur_r_muscle_paths.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/patella_l_muscle_paths.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/patella_r_muscle_paths.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/tibia_l_muscle_paths.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/tibia_r_muscle_paths.csv', index_col=0)])
mscles_df = mscles_df.set_index('name')

### Import Transformed Wrapping Surfaces

In [11]:
wrp_df  = pd.concat([pd.read_csv('tps_warping_results/after_MRI/pelvis_wrap_translations.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/femur_l_wrap_translations.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/femur_r_wrap_translations.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/tibia_l_wrap_translations.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/tibia_r_wrap_translations.csv', index_col=0)])
wrp_df = wrp_df.set_index('name')

### Import transformed Skin Markers

In [12]:
skin_df = pd.concat([pd.read_csv('tps_warping_results/after_MRI/pelvis_skin_markers.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/femur_l_skin_markers.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/femur_r_skin_markers.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/tibia_l_skin_markers.csv', index_col=0),
            pd.read_csv('tps_warping_results/after_MRI/tibia_r_skin_markers.csv', index_col=0)])
skin_df = skin_df.set_index('name')
skin_df.to_csv('tps_warping_results/after_MRI/skin_transformed.csv')

### Parse current scaled model

In [13]:
tree=ET.parse(scaled_model)
root = tree.getroot()


### Update reference geometry

In [14]:
ref_geom_dic = {'body': [], 'Mesh name' : [], 'Mesh file' : []}

ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\r_pelvis.stl'))
ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_2"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\l_pelvis.stl'))
ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_3"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\sacrum.stl'))

ref_geom_dic['body'].append("femur_r"); ref_geom_dic['Mesh name'].append("femur_r_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\femur_r.stl'))
ref_geom_dic['body'].append("femur_l"); ref_geom_dic['Mesh name'].append("femur_l_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\femur_l.stl'))

ref_geom_dic['body'].append("tibia_r"); ref_geom_dic['Mesh name'].append("tibia_r_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\tibia_r.stl'))
ref_geom_dic['body'].append("tibia_r"); ref_geom_dic['Mesh name'].append("tibia_r_geom_2"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\fibula_r.stl'))

ref_geom_dic['body'].append("tibia_l"); ref_geom_dic['Mesh name'].append("tibia_l_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\tibia_l.stl'))
ref_geom_dic['body'].append("tibia_l"); ref_geom_dic['Mesh name'].append("tibia_l_geom_2"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\fibula_l.stl'))

ref_geom_dic['body'].append("patella_r"); ref_geom_dic['Mesh name'].append("patella_r_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\patella_r.stl'))
ref_geom_dic['body'].append("patella_l"); ref_geom_dic['Mesh name'].append("patella_l_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\patella_l.stl'))

In [15]:
ref_geom_df = pd.DataFrame(ref_geom_dic).set_index('Mesh name')

In [16]:
for Mesh in root.iter('Mesh'):
    point = Mesh.attrib['name']
    if point in ref_geom_df.index:
        new_text = ref_geom_df.loc[point, 'Mesh file']
        # print(point, new_text)
        Mesh.find('mesh_file').text = new_text
        Mesh.find('scale_factors').text = '1 1 1'
    else: pass

### Update muscle paths

In [17]:
muscle_path_names = []
muscle_path_old = []
muscle_path_new = []

for PathPoint in root.iter('PathPoint'):
    point = PathPoint.attrib['name']   
    if point in mscles_df.index:      
        
        location = mscles_df.loc[point, 'location']
        new_text = location[1:-1]
        muscle_path_names.append(point)
        muscle_path_new.append(new_text)
        muscle_path_old.append(PathPoint.find('location').text)

        PathPoint.find('location').text = new_text
    else: pass

In [18]:
muscle_path_new_df = pd.DataFrame([[float(i) for i in point.split()] for point in muscle_path_new], index=muscle_path_names, columns=['x','y','z'])
muscle_path_old_df = pd.DataFrame([[float(i) for i in point.split()] for point in muscle_path_old], index=muscle_path_names, columns=['x','y','z'])

muscle_path_diff_df = muscle_path_old_df - muscle_path_new_df
muscle_path_diff_df['d'] = (muscle_path_diff_df['x']**2+muscle_path_diff_df['y']**2+muscle_path_diff_df['z']**2)**0.5

### Update muscle wrapping surfaces

In [19]:
wrap_cyl_name = []
wrap_transl_old = []
wrap_transl_new = []
wrap_radius_old = []
wrap_radius_new = []

In [26]:
for WrapCylinder in root.iter('WrapCylinder'):
    point = WrapCylinder.attrib['name']
    wrap_cyl_name.append(point)

    if point in wrp_df.index:
        translation = wrp_df.loc[point, 'location']
        wrap_transl_new.append(translation)
        wrap_transl_old.append(WrapCylinder.find('translation').text)
        # Defensive: check if translation is a pandas Series (should not be, but just in case)
        if isinstance(translation, pd.Series):
            print(f'{translation} is a pandas Series, selecting the first row')
            translation = translation.iloc[0]
            
        WrapCylinder.find('translation').text = translation[1:-1]
        
        

    else: pass

name
Flex_at_femhead_r    [-0.00505375816651461  0.00081413088808978  0.00936748614800365]
Flex_at_femhead_r    [-0.00265201563865854 -0.00068066765426039  0.01179596002901365]
Name: location, dtype: object is a pandas Series, selecting the first row
name
Flex_at_femhead_r    [-0.00505375816651461  0.00081413088808978  0.00936748614800365]
Flex_at_femhead_r    [-0.00265201563865854 -0.00068066765426039  0.01179596002901365]
Name: location, dtype: object is a pandas Series, selecting the first row


### Update Translation of joint centres

In [27]:
torso_origin_in_pelvis = markers_df.loc['torso_origin_in_pelvis', 'location']

femur_r_center_in_pelvis = markers_df.loc['femur_r_center_in_pelvis', 'location']
femur_l_center_in_pelvis = markers_df.loc['femur_l_center_in_pelvis', 'location']
knee_l_center_in_femur = markers_df.loc['knee_l_center_in_femur_l', 'location']
knee_r_center_in_femur = markers_df.loc['knee_r_center_in_femur_r', 'location']
patella_l_center_in_femur = markers_df.loc['patella_l_in_femur_l', 'location']
patella_r_center_in_femur = markers_df.loc['patella_r_in_femur_r', 'location']

talus_l_center_in_tibia = markers_df.loc['talus_l_center_in_tibia', 'location']
talus_r_center_in_tibia = markers_df.loc['talus_r_center_in_tibia', 'location']

In [28]:
joint_centers_names = []
joint_centers_new = []
joint_centers_old = []

for joint in root.iter('CustomJoint'):
    
    name = joint.attrib['name']
    if joint.attrib['name'] ==  "back":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = torso_origin_in_pelvis[1:-1]
                joint_centers_names.append("back")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text

            else: pass

    if joint.attrib['name'] ==  "hip_r":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":                
                new_text = femur_r_center_in_pelvis[1:-1]
                joint_centers_names.append("hip_r")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)           
                frame.find('translation').text = new_text

            else: pass
    
    elif joint.attrib['name'] ==  "hip_l":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = femur_l_center_in_pelvis[1:-1]
                joint_centers_names.append("hip_l")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text
            else: pass
            
    elif joint.attrib['name'] ==  "walker_knee_r":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "femur_r_offset":
                new_text = knee_r_center_in_femur[1:-1]
                joint_centers_names.append("walker_knee_r")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text

            #if frame.attrib['name'] == "tibia_r_offset":
            #    new_text = f"{walker_knee_r_tibia_r_offset[0]}, {walker_knee_r_tibia_r_offset[1]}, {walker_knee_r_tibia_r_offset[2]}"
            #    frame.find('translation').text = new_text
            else: pass    
    
    elif joint.attrib['name'] ==  "walker_knee_l":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "femur_l_offset":
                new_text = knee_l_center_in_femur[1:-1]
                joint_centers_names.append("walker_knee_l")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text

    #        if frame.attrib['name'] == "tibia_l_offset":
    #            new_text = f"{walker_knee_l_tibia_l_offset[0]}, {walker_knee_l_tibia_l_offset[1]}, {walker_knee_l_tibia_l_offset[2]}"
    #            frame.find('translation').text = new_text
            else: pass

    elif joint.attrib['name'] ==  "patellofemoral_r":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "femur_r_offset":               
                new_text = knee_r_center_in_femur[1:-1]
                joint_centers_names.append("patellofemoral_r")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text
           else: pass
    
    elif joint.attrib['name'] ==  "patellofemoral_l":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "femur_l_offset":              
                new_text = knee_l_center_in_femur[1:-1]
                joint_centers_names.append("patellofemoral_l")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text
           else: pass

for joint in root.iter('PinJoint'):   
    name = joint.attrib['name']
    if joint.attrib['name'] ==  "ankle_r":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "tibia_r_offset":              
                new_text = talus_r_center_in_tibia [1:-1]
                joint_centers_names.append("ankle_r")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text
           else: pass
    
    elif joint.attrib['name'] ==  "ankle_l":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "tibia_l_offset":              
                new_text = talus_l_center_in_tibia [1:-1]              
                joint_centers_names.append("ankle_l")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)              
                frame.find('translation').text = new_text
           else: pass
    else: pass

### Update Markers for Joint Positions in Parent

In [29]:
m_names = ['torso_origin_in_pelvis','femur_l_center_in_pelvis', 'femur_r_center_in_pelvis',  'knee_l_center_in_femur_l', 'patella_l_in_femur_l', 'knee_r_center_in_femur_r', 'patella_r_in_femur_r']

for marker in root.iter('Marker'):
    name = marker.attrib['name']
    if name == 'ankle_l_tibia_l_offset':
        old_text = marker.find('location').text
        new_text = markers_df.loc['talus_l_center_in_tibia', 'location']
        marker.find('location').text = new_text[1:-1]
    elif name == 'ankle_r_tibia_r_offset':
        old_text = marker.find('location').text
        new_text = markers_df.loc['talus_r_center_in_tibia', 'location']
        marker.find('location').text = new_text[1:-1]
    elif name in m_names:
        old_text = marker.find('location').text
        new_text = markers_df.loc[name, 'location']
        marker.find('location').text = new_text[1:-1]

### Update Skin Markers

In [30]:
for marker in root.iter('Marker'):
    name = marker.attrib['name']
    if name in list(skin_df.index):
        old_text = marker.find('location').text
        new_text = skin_df.loc[name, 'location']
        marker.find('location').text = new_text[1:-1]
    else: pass

### Export transformed model

In [24]:
print(tree)

In [31]:
# Debug: Check the types and values of problematic variables
print("Type of patella_l_in_femur:", type(patella_l_in_femur))
print("Value of patella_l_in_femur:", patella_l_in_femur)
print()
print("Type of patella_r_in_femur:", type(patella_r_in_femur))
print("Value of patella_r_in_femur:", patella_r_in_femur)

Type of patella_l_in_femur: <class 'pandas.core.series.Series'>
Value of patella_l_in_femur: body                                                                 femur_l
location    [ 0.04363238677200714 -0.4128031117130943   0.00217404444689124]
Name: patella_l_in_femur_l, dtype: object

Type of patella_r_in_femur: <class 'pandas.core.series.Series'>
Value of patella_r_in_femur: body                                                                 femur_r
location    [ 0.04543757553456587 -0.41757370067727007 -0.002547464627669  ]
Name: patella_r_in_femur_r, dtype: object


In [35]:
import xml.etree.ElementTree as ET
import pandas as pd

def find_pandas_in_tree(root_element):
    """
    Iterates through an ElementTree and finds elements where the .text
    or an attribute value is a pandas Series or DataFrame.

    Args:
        root_element: The root element of the XML tree to search.
    """
    print("🕵️  Searching for pandas objects in the XML tree...")
    found_issue = False
    # root_element.iter() visits every element in the tree
    for element in root_element.iter():
        # 1. Check the element's text content
        if isinstance(element.text, (pd.Series, pd.DataFrame)):
            found_issue = True
            print(f"🐛 PROBLEM FOUND in element: <{element.tag}>")
            print(f"   - The .text content is a pandas object.")
            print(f"   - Value: {element.text.to_string()}") # Use .to_string() for clean printing
            
            element.text = element.text.to_string()  # Convert to string for XML compatibility
            
        # 2. Check all of the element's attributes
        for key, value in element.attrib.items():
            if isinstance(value, (pd.Series, pd.DataFrame)):
                found_issue = True
                print(f"🐛 PROBLEM FOUND in element: <{element.tag}>")
                print(f"   - The attribute '{key}' is a pandas object.")
                print(f"   - Value: {value.to_string()}")
                
                # if found_issue make them a string
                element.set(key, value.to_string())                
    
    if not found_issue:
        print("✅ No pandas objects found. The tree looks clean!")
        
    return root_element

# --- HOW TO USE IT ---

# Assume 'tree' is your ElementTree object from your code
# For example: 

# Before you call tree.write(), run the diagnostic function:
root = find_pandas_in_tree(root) 

# Your original failing line:
# tree.write(new_model_name)

🕵️  Searching for pandas objects in the XML tree...
🐛 PROBLEM FOUND in element: <translation>
   - The .text content is a pandas object.
   - Value: Series([], )
🐛 PROBLEM FOUND in element: <translation>
   - The .text content is a pandas object.
   - Value: Series([], )


In [32]:
tree.write(new_model_name)
print(f'data saved in {new_model_name}')

data saved in e:\DataFolder\AlexP\models\athlete_03\001bis\P01\model_update\4_tps-bones-muscles-updated/P01bis_tps_transformed.osim


## Scale muscle fibres

In [ ]:
# from fibre_scale_script import *

In [ ]:
# # define models
# generic_model = '../templates/RajagopalModified_generic_copy.osim'
# model_to_update = f'4_tps-bones-muscles-updated/{ind}_tps_transformed.osim'
# updated_model = f'4_tps-bones-muscles-updated/{ind}_tps_fibres_updated.osim'

# # run scripts
# osimModel_opt, SimInfo = optimMuscleParams(generic_model, model_to_update, 2, '7_tps-fibres-updated/logging')

# # printing the optimized model
# osimModel_opt.printToXML(updated_model)

## Fit skin markers

In [33]:
new_model_name

'e:\\DataFolder\\AlexP\\models\\athlete_03\\001bis\\P01\\model_update\\4_tps-bones-muscles-updated/P01bis_tps_transformed.osim'

In [34]:
## Fit Skin Markers
# <model_file>
model = new_model_name # f'4_tps-bones-muscles-updated/{}_tps_fibres_updated.osim' #

# <marker_set_file>
path_to_generic_marker_set = '4_tps-bones-muscles-updated/markers.xml' #'unassigned' #'../templates/scaling_EXP_Markers_to_Scale_final_3.xml'

# <time_range>'
time_range = f' 0 {list(static_df.loc[static_df.shape[0], "Time"])[0] }'

# <output_scale_file>
path_to_model = '4_tps-bones-muscles-updated'
output_scale_file = os.path.join(path_to_model, 'marker_placer.txt')
output_scaling_settings = os.path.join(path_to_model, f'marker_placer_{ind}.xml')
output_model_file = os.path.join(path_to_model, f'{ind}_tps_fibres_skin_updated.osim')

In [35]:
output_scaling_settings

'4_tps-bones-muscles-updated\\marker_placer_P01bis.xml'

In [36]:
# parse the .xml with markers in body frames
scaling_tree=ET.parse("../templates/marker_placer.xml")
scaling_root = scaling_tree.getroot()

for generic_model in scaling_root.iter('model_file'):
    generic_model.text = model
for generic_marker_set in scaling_root.iter('marker_set_file'):
    generic_marker_set.text = path_to_generic_marker_set

for exp_markers in scaling_root.iter('marker_file'):
    exp_markers.text = experimental_markers

for time in scaling_root.iter('time_range'):
    time.text = time_range
for output in scaling_root.iter('output_model_file'):
    output.text = output_model_file
for scale in scaling_root.iter('output_scale_file'):
    scale.text = output_scale_file
for generic_marker_set in scaling_root.iter('marker_set_file'):
    generic_marker_set.text = path_to_generic_marker_set

for time in scaling_root.iter('time_range'):
    time.text = time_range
for output in scaling_root.iter('output_model_file'):
    output.text = output_model_file
for scale in scaling_root.iter('output_scale_file'):
    scale.text = output_scale_file

In [38]:
scaling_tree.write(output_scaling_settings)
print(f'Scaling settings saved to {output_scaling_settings}')

Scaling settings saved to 4_tps-bones-muscles-updated\marker_placer_P01bis.xml


In [ ]:
# # Create a ScaleTool object using the configuration file.
# scale_tool = osim.ScaleTool(output_scaling_settings)

# # Print some information of the config file to test everything is correct.
# print("Name:", scale_tool.getName())
# print("Subject Mass:", scale_tool.getSubjectMass())
# print("Subject Height:", scale_tool.getSubjectHeight())
# print("Notes:", scale_tool.getPropertyByName("notes").toString())
# print()
# # Get model marker file name.
# generic_model_maker = scale_tool.getGenericModelMaker()
# print("Marker Set File Name:", generic_model_maker.getMarkerSetFileName())
# print("Model File Name:", generic_model_maker.getModelFileName())
# print()


In [ ]:
# scale_tool.run()

In [39]:
from simFunctions import runProgram

cmdprog = 'opensim-cmd'
cmdtool = 'run-tool'
cmdfile = output_scaling_settings
cmdfull = [cmdprog, cmdtool, cmdfile]
rc = runProgram(cmdfull)

c:\Users\Bas\miniconda3\envs\py311\Lib\subprocess.py:1016: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


[info] Preparing to run ScaleTool.

[info] Processing subject Rajagopal-generic-scaled-scaled...

[info] Step 1: Loading generic model

[info] Loaded model Rajagopal-generic-scaled from file e:\DataFolder\AlexP\models\athlete_03\001bis\P01\model_update\4_tps-bones-muscles-updated\P01bis_tps_transformed.osim

[warning] Couldn't find file 'bones\r_pelvis.stl'.

[warning] Couldn't find file 'bones\l_pelvis.stl'.

[warning] Couldn't find file 'sacrum.vtp'.

[warning] Couldn't find file 'bones\femur_r.stl'.

[warning] Couldn't find file 'bones\tibia_r.stl'.

[warning] Couldn't find file 'bones\fibula_r.stl'.

[warning] Couldn't find file 'bones\patella_r.stl'.

[warning] Couldn't find file 'r_talus.vtp'.

[warning] Couldn't find file 'r_foot.vtp'.

[warning] Couldn't find file 'r_bofoot.vtp'.

[warning] Couldn't find file 'bones\femur_l.stl'.

[warning] Couldn't find file 'bones\tibia_l.stl'.

[warning] Couldn't find file 'bones\fibula_l.stl'.

[warning] Couldn't find file 'bones\patella_l.

## Update Wrapping Surfaces

In [40]:
from wrap_scripts import *

model_to_update = f'4_tps-bones-muscles-updated/{ind}_tps_fibres_skin_updated.osim'
updated_model = f'4_tps-bones-muscles-updated/{ind}_tps_fibres_skin_wrp_updated.osim'
# function to extract muscle paths, wrapping surfaces info and joint info from an osim model
# returns three dataframes: muscles, surfaces and joints
fix_wraps = FixWraps(model_to_update)

FileNotFoundError: [Errno 2] No such file or directory: '4_tps-bones-muscles-updated/P01bis_tps_fibres_skin_updated.osim'

In [ ]:
wrp_adjust_dict = fix_wraps.collect_wrp_details()
wrp_adjust_df = pd.DataFrame.from_dict(wrp_adjust_dict)
wrp_adjust_df.set_index(pd.Index(['body', 'transl', 'rad', 'point_label', 'point', 'point_body', 'joint_location', 'joint_location+point', 'dist_to_transl',  'dist']), inplace = True)


In [ ]:
wrp_adjust_df

In [ ]:
#print points whose location is too far into radius for an automatic adjustment
manual = []
for col in wrp_adjust_df.columns:
    if wrp_adjust_df.loc['dist', col] < wrp_adjust_df.loc['rad', col]-0.01:
        print(col, ':', wrp_adjust_df.loc['point_label', col])
        manual.append(col)

# ignore patella points for now
# adjust other points or wrapping surfaces manually for the input model and run the script again

In [ ]:
# # update model
tree_model = ET.parse(model_to_update)
root = tree_model.getroot()

wrp_to_change = [i for i in wrp_adjust_df.columns if i not in manual]

for WrapCylinder in root.iter('WrapCylinder'):
    name = WrapCylinder.attrib['name']
    if name in wrp_to_change:
        print(name)
        radius = wrp_adjust_df.loc['rad', name]
        print('old_radius', radius)
        new_rad = wrp_adjust_df.loc['dist', name]
        print('new_radius', new_rad)
        radius_new_text = str(new_rad-0.01)
        WrapCylinder.find('radius').text = radius_new_text

tree_model.write(updated_model)

<div class="alert alert-block alert-info">
<b>IN OPENSIM GUI</b> 
Open the updated model. Activate wrapping surfaces in the list od surfaces whose adjustment is more than 0.02 above and related muscles. Make sure displayed surfaces do not overlap -- switch on one at a time. In the list of muscles, highlight and display the related muscle. In the properties window for that muscle, click on '...' for the muscle path, choose (by cklicking the left square) the listed via point. Interact with the 3D view to pull the via point just outside of the wrapping surface.

Once finished with the list, <b> save the model </b>.
</div>